<a href="https://colab.research.google.com/github/felladib/Breast-Cancer-Classification-with-Deep-Learning/blob/main/CnacerBreastClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Classification du cancer du sein – À propos du projet Python**
Dans ce projet en python, nous allons construire un classificateur pour s'entraîner sur 80 % d'un ensemble de données d'images histologiques du cancer du sein. Sur ce montant, nous conserverons 10 % des données pour validation. En utilisant Keras, nous définirons un CNN (**Convolutional Neural Network**) , l'appellerons CancerNet et l'entraînerons sur nos images. Nous dériverons ensuite une matrice de confusion pour analyser les performances du modèle.

IDC est un carcinome canalaire invasif ; un cancer qui se développe dans un canal galactophore et envahit le tissu mammaire fibreux ou adipeux à l'extérieur du canal ; c'est la forme de cancer du sein la plus courante et représente 80 % de tous les diagnostics de cancer du sein. Et l'histologie est l'étude de la structure microscopique des tissus.


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import matplotlib
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adagrad
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cancernet.cancernet import CancerNet
from cancernet import config
from imutils import paths


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"felladb","key":"ef42a10a3bfbf2e9e52b86ea65b6f0ca"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

 99% 3.08G/3.10G [00:15<00:00, 271MB/s]
100% 3.10G/3.10G [00:15<00:00, 211MB/s]


In [ ]:
!unzip breast-histopathology-images.zip -d data  # Assurez-vous que le nom du fichier zip est correct
# le -d data indique que vous voulez extraire le contenu de l'archive breast-histopathology-images.zip dans un répertoire nommé data

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: data/ID

In [ ]:
TRAIN_SPLIT = 0,8
VAL_SPLIT = 0,1

# **build_dataset**
This will split our dataset into training, validation, and testing sets in the ratio mentioned above- 80% for training (of that, 10% for validation) and 20% for testing. With the **ImageDataGenerator** from Keras, we will extract batches of images to avoid making space for the entire dataset in memory at once.

l'utilisation de **ImageDataGenerator** de Keras et la mention des lots (ou "batches") fait référence à une technique courante en apprentissage en profondeur pour traiter de grands ensembles de données, en particulier des ensembles d'images.

Lorsque vous travaillez avec des ensembles de données d'images de grande taille, charger toutes les images en mémoire en une seule fois peut être excessif en termes de mémoire RAM. Pour éviter cela, on utilise la technique du "batch processing" (traitement par lots), où les données sont divisées en lots plus petits, et seulement un lot est chargé en mémoire à la fois pour être traité.

Tout à fait, vous avez saisi l'essentiel du fonctionnement de **ImageDataGenerator**. Cette méthode est spécialement utile lorsque vous travaillez avec des ensembles de données volumineux, tels que des ensembles d'images.

Avec **ImageDataGenerator** de Keras, on divise et prétraite les données image par image, ou en lots (batches), pour les charger dans le modèle d'apprentissage en profondeur.

Voici comment cela fonctionne de manière générale :

**Prétraitement par lots** : ImageDataGenerator permet de charger des lots d'images (par lot, par exemple, 32 images à la fois) à partir d'un répertoire. Ces images peuvent être prétraitées en temps réel (redimensionnées, normalisées, augmentées) à chaque itération.

**Entraînement par lots** : Une fois prétraité, chaque lot (ou batch) d'images est ensuite utilisé pour entraîner le modèle. Le modèle passe par une itération d'entraînement avec chaque batch.

**Itérations successives **: Ce processus se répète pour chaque lot disponible dans l'ensemble de données. Les lots sont traités successivement pour l'entraînement du modèle.

Cette approche permet d'économiser de la mémoire en ne chargeant pas toutes les images en même temps, mais plutôt en les traitant par lots, ce qui est particulièrement avantageux pour les ensembles de données volumineux.

En résumé, ImageDataGenerator facilite le prétraitement des données d'images par lots successifs, permettant ainsi l'entraînement du modèle sur de grands ensembles de données sans nécessiter un espace mémoire excessif.








In [ ]:
from imutils import paths
import random, shutil, os

# Répertoire où vous avez extrait les données
data_path = '/content/data'  # Adapter le chemin selon l'emplacement de vos données

# os.listdir() retourne une liste des noms de tous les éléments
originalPaths = list(paths.list_images(data_path)) #  Cette ligne utilise la fonction list_images fournie par imutils.paths pour lister tous les chemins des images présentes dans le répertoire data_path. Ces chemins sont stockés dans la variable originalPaths sous forme de liste.
# paths.list_images() (de la bibliothèque imutils.paths) est une fonction spécifique qui retourne une liste des chemins des fichiers images (par exemple, avec des extensions comme .jpg, .png, etc.) présents dans le répertoire spécifié. Cette fonction filtre automatiquement les fichiers pour ne retourner que les fichiers image.

random.seed(7)
random.shuffle(originalPaths) # Cette ligne mélange aléatoirement l'ordre des chemins des images dans originalPaths. Cela est utile pour s'assurer que l'ensemble de données n'a pas de structure particulière (comme toutes les images du même label groupées ensemble), ce qui pourrait biaiser l'apprentissage.

train_split = 0.8
val_split = 0.1

index_train = int(len(originalPaths) * train_split)
trainPaths = originalPaths[:index_train]
testPaths = originalPaths[index_train:]

index_val = int(len(trainPaths) * val_split)
valPaths = trainPaths[:index_val]
trainPaths = trainPaths[index_val:]

datasets = [
    ("training", trainPaths, '/content/training'),
    ("validation", valPaths, '/content/validation'),
    ("testing", testPaths, '/content/testing')
]

for (setType, originalPaths, basePath) in datasets:
    print(f'Building {setType} set')

    if not os.path.exists(basePath): # si le path n'exist pas cree le .
        os.makedirs(basePath)

    for path in originalPaths:  #Création des répertoires de classes pour chaque ensemble de données :
        file = path.split(os.path.sep)[-1] #9383_idx5_x901_y1751_class0.png
        label = file[-5:-4] #zero

        labelPath = os.path.join(basePath, label) #/content/training/0
        if not os.path.exists(labelPath):
            os.makedirs(labelPath) #sinn cree ce path

        newPath = os.path.join(labelPath, file) #/content/training/0/9383_idx5_x901_y1751_class0.png.
        shutil.copy2(path, newPath) #shutil.copy2(path, newPath): Cette ligne copie effectivement le fichier d'image original (situé à path) vers le nouveau chemin (newPath)


Building training set
Building validation set
Building testing set


# **Build the Model**
The network we’ll build will be a CNN (Convolutional Neural Network) and call it CancerNet. This network performs the following operations:

Use 3×3 CONV filters
Stack these filters on top of each other
Perform max-pooling
Use depthwise separable convolution (more efficient, takes up less memory)

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [ ]:
class CancerNet(tf.keras.Model):
    def __init__(self, dropout=0, width=0, height=0, depth=0, classes=0, filter_size=32):
        super(CancerNet, self).__init__()

        self.shape = (height, width, depth)
        self.channelDim = -1

        if K.image_data_format() == "channels_first":
            self.shape = (depth, height, width)
            self.channelDim = 1

        self.Dropout_layers = []
        self.SeprConv_layers = []
        self.BatchNorm_layers = []
        self.maxPol_layers = []
        self.Activ_layers = []

        for i in range(3):
            for j in range(i + 1):
                self.SeprConv_layers.append(
                    SeparableConv2D(filter_size * 2**i, (3, 3), padding='same', input_shape=self.shape)
                )
                self.Activ_layers.append(Activation('relu'))
                self.BatchNorm_layers.append(BatchNormalization())

            self.maxPol_layers.append(MaxPooling2D(pool_size=(2, 2)))
            self.Dropout_layers.append(Dropout(0.25))

        self.flatten = Flatten()
        self.dense1 = Dense(256)
        self.activation1 = Activation('relu')

        self.dropout = Dropout(0.5)
        self.Norm = BatchNormalization()
        self.dense2 = Dense(classes)
        self.activation2 = Activation('softmax')

    def call(self, inputs):
        x = inputs
        cmpt = 0
        for i in range(3):
            for j in range(i + 1):
                x = self.SeprConv_layers[cmpt](x)
                x = self.Activ_layers[cmpt](x)
                x = self.BatchNorm_layers[cmpt](x)
                cmpt += 1
            x = self.maxPol_layers[i](x)
            x = self.Dropout_layers[i](x)

        x = self.flatten(x)
        x = self.dense1(x)
        x = self.activation1(x)
        x = self.Norm(x)
        x = self.dropout(x)
        x = self.dense2(x)
        return self.activation2(x)


# **Train_model**

In [ ]:
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adagrad
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cancernet.cancernet import CancerNet
from cancernet import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

j and i 0 0
j and i 0 1
j and i 1 1
j and i 0 2
j and i 1 2
j and i 2 2


In [ ]:
num_epoch = 40
init_lr = 1e-2
BS=32

In [ ]:
# Chemins vers les répertoires d'entraînement, de validation et de test
train_path = '/content/training'
val_path = '/content/validation'
test_path = '/content/testing'

# Compter le nombre d'images dans chaque ensemble
lenTrain = sum(len(files) for _, _, files in os.walk(train_path))
lenVal = sum(len(files) for _, _, files in os.walk(val_path))
lenTest = sum(len(files) for _, _, files in os.walk(test_path))

In [ ]:
NUM_EPOCHS=40; INIT_LR=1e-2; BS=32
trainPaths=list(paths.list_images(config.TRAIN_PATH))
lenTrain=len(trainPaths)
lenVal=len(list(paths.list_images(config.VAL_PATH)))
lenTest=len(list(paths.list_images(config.TEST_PATH)))
trainLabels=[int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels=np_utils.to_categorical(trainLabels)
classTotals=trainLabels.sum(axis=0)
classWeight=classTotals.max()/classTotals
trainAug = ImageDataGenerator(
  rescale=1/255.0,
  rotation_range=20,
  zoom_range=0.05,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.05,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode="nearest")
valAug=ImageDataGenerator(rescale=1 / 255.0)
trainGen = trainAug.flow_from_directory(
  config.TRAIN_PATH,
  class_mode="categorical",
  target_size=(48,48),
  color_mode="rgb",
  shuffle=True,
  batch_size=BS)
valGen = valAug.flow_from_directory(
  config.VAL_PATH,
  class_mode="categorical",
  target_size=(48,48),
  color_mode="rgb",
  shuffle=False,
  batch_size=BS)
testGen = valAug.flow_from_directory(
  config.TEST_PATH,
  class_mode="categorical",
  target_size=(48,48),
  color_mode="rgb",
  shuffle=False,
  batch_size=BS)
model=CancerNet.build(width=48,height=48,depth=3,classes=2)
opt=Adagrad(lr=INIT_LR,decay=INIT_LR/NUM_EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
M=model.fit_generator(
  trainGen,
  steps_per_epoch=lenTrain//BS,
  validation_data=valGen,
  validation_steps=lenVal//BS,
  class_weight=classWeight,
  epochs=NUM_EPOCHS)
print("Now evaluating the model")
testGen.reset()
pred_indices=model.predict_generator(testGen,steps=(lenTest//BS)+1)
pred_indices=np.argmax(pred_indices,axis=1)
print(classification_report(testGen.classes, pred_indices, target_names=testGen.class_indices.keys()))
cm=confusion_matrix(testGen.classes,pred_indices)
total=sum(sum(cm))
accuracy=(cm[0,0]+cm[1,1])/total
specificity=cm[1,1]/(cm[1,0]+cm[1,1])
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
print(cm)
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N), M.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), M.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), M.history["acc"], label="train_acc")
plt.plot(np.arange(0,N), M.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on the IDC Dataset")
plt.xlabel("Epoch No.")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('plot.png')